In [2]:
import gymnasium as gym
import panda_gym
from sb3_contrib import TQC
from huggingface_sb3 import load_from_hub
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
import time
import numpy as np
from panda_gym.utils import distance
import random

In [3]:
chk = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="tqc-PandaPickAndPlace-v3.zip")
stats = load_from_hub(repo_id="BanUrsus/tqc-PandaPickAndPlace-v3", filename="vec_normalize.pkl")
   
env = gym.make('PandaPickAndPlace-v3', render_mode="human", autoreset=False)

sim = env.sim
robot = env.robot
task = env.task

env = DummyVecEnv([lambda: env])
env = VecNormalize.load(stats, env)
env.training = False
env.norm_reward = False

pybullet build time: Nov 28 2023 23:48:36
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.robot to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.robot` for environment variables or `env.get_wrapper_attr('robot')` that will search the reminding wrappers.
  logger.warn(
/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.task to get variables from other wrappers is deprecated and will be removed in v1.0, to get this va

In [4]:
model = TQC.load(chk,env) 
env = model.env
obs = env.reset()

q1 = np.array([0.15, 0.15 , 0.0])
q2 = np.array([0.15, -0.15, 0.0])
q3 = np.array([-0.15, 0.15, 0.0])
q4 = np.array([-0.15, -0.15, 0.0])

/home/frosty/Programs/Python/cse598/.venv/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


In [5]:
sim.create_box(
    body_name="q1",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q1,
    rgba_color=np.array([1.0, 0.0, 0.0, 1.0]),
)

In [6]:
sim.create_box(
    body_name="q2",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q2,
    rgba_color=np.array([0.5, 0.5, 0.0, 1.0]),
)

In [7]:
sim.create_box(
    body_name="q3",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q3,
    rgba_color=np.array([0.0, 0.0, 1.0, 1.0]),
)

In [8]:
sim.create_box(
    body_name="q4",
    half_extents=np.ones(3) * 0.04 / 2,
    mass=1.0,
    position=q4,
    rgba_color=np.array([0.0, 0.5, 0.5, 1.0]),
)

In [6]:
def is_gripper_open(finger_width):
    return finger_width > 0.02

def is_gripper_correctly_oriented(ee_position, block_position, z_threshold=0.025):
    return abs(ee_position[2] - block_position[2]) <= z_threshold 

def is_block_grasped(finger_width, proximity):
    return finger_width <= 0.08 and finger_width >= 0.02 and proximity < 0.05  

def estimate_finger_positions(ee, fw):
    hw = fw / 2
    lf = np.array([ee[0] - hw, ee[1], ee[2]])
    rf = np.array([ee[0] + hw, ee[1], ee[2]])
    return lf, rf

def is_block_between_fingers(bp, lf, rf):
    return (min(lf[0], rf[0]) <= bp[0] <= max(lf[0], rf[0])) and (min(lf[1], rf[1]) <= bp[1] <= max(lf[1], rf[1]))
    
# TODO
# Change Defn of is block is being gripped 
    

def is_sliding(block_position, goal_position, z_threshold=0.02):
    # Check if the block and goal are on the same z-plane within a threshold
    
    return abs(block_position[2] - goal_position[2]) <= z_threshold

In [22]:
random_states = []
solved_states = []

def is_grasped(fw):
    robot_idx = sim._bodies_idx["panda"]
    block_idx = sim._bodies_idx["object"]
    
    gripperState = sim.physics_client.getLinkState(robot_idx, robot.ee_link)
    gripperPos = gripperState[0]
    gripperOrn = gripperState[1]
    blockPos, blockOrn = sim.physics_client.getBasePositionAndOrientation(block_idx)

    invGripperPos, invGripperOrn = sim.physics_client.invertTransform(gripperPos, gripperOrn)
    gripperMat = sim.physics_client.getMatrixFromQuaternion(gripperOrn)
    dir0 = [gripperMat[0], gripperMat[3], gripperMat[6]]
    dir1 = [gripperMat[1], gripperMat[4], gripperMat[7]]
    dir2 = [gripperMat[2], gripperMat[5], gripperMat[8]]

    gripperEul = sim.physics_client.getEulerFromQuaternion(gripperOrn)
    #print("gripperEul")
    #print(gripperEul)
    blockPosInGripper, blockOrnInGripper = sim.physics_client.multiplyTransforms(invGripperPos, invGripperOrn,
                                                                blockPos, blockOrn)
    projectedBlockPos2D = [blockPosInGripper[0], blockPosInGripper[1]]
    blockEulerInGripper = sim.physics_client.getEulerFromQuaternion(blockOrnInGripper)
    #print("projectedBlockPos2D")
    #print(projectedBlockPos2D)
    #print("blockEulerInGripper")
    #print(blockEulerInGripper)

    return abs(blockPosInGripper[0]) < 0.02 and abs(blockPosInGripper[1]) < 0.02 and abs(blockPosInGripper[2]) < 0.05  and fw < 0.08 and fw > 0.02




while len(random_states) < 10000 or len(solved_states) < 1000:
    action, _states = model.predict(obs, deterministic=True)
    # print(action)
    obs, reward, dones, info = env.step(action)

    # gather data
    ee = robot.get_ee_position()
    fw = robot.get_fingers_width()
    block = task.get_achieved_goal()
    goal = np.array(sim.get_base_position("target"))

    lf, rf = estimate_finger_positions(ee, fw)

   #print(block, ee)
    #print("block between: ", is_block_between_fingers(block, lf,rf))
    # predicates
    distance_to_block = distance(ee, block)
    gripper_open = is_gripper_open(fw)
    gripper_correctly_oriented = is_gripper_correctly_oriented(ee,block)
    block_grasped = is_block_grasped(fw, distance_to_block)
    sliding = is_sliding(block,goal)
    
    object_velocity = sim.get_base_velocity("object")

    robot_idx = sim._bodies_idx["panda"]
    block_idx = sim._bodies_idx["object"]
    print(len(sim.physics_client.getClosestPoints(block_idx, robot_idx, distance=0, linkIndexB=9)) > 0 and fw > 0.01)#robot.ee_link))
    #print("is grasped?", is_grasped(fw))#block_grasped and np.sum(object_velocity) > 0)
    # abstract state
    state = {
        "effector_pos": ee,
        "finger_width": fw,
        "gripper_open": gripper_open,
        "gripper_orientation_correct": gripper_correctly_oriented,
        "block_grasped": block_grasped,
        "d_q1": distance(ee, q1),
        "d_q2": distance(ee, q2),
        "d_q3": distance(ee, q3),
        "d_q4": distance(ee, q4),
        "distance_to_block": distance_to_block,
        "block_to_goal": distance(block, goal),
        "is_grasping": fw > 0.02 and fw < 0.04,
        "is_closed": fw < 0.01,
        "sliding": sliding,
        "finished": dones[0]
    }

    time.sleep(1)

    if dones[0]:
        solved_states.append(state)
        obs = env.reset()
    else:
        if random.randint(0, 10) == 0:
            random_states.append(state)

print(f"finished: {len(random_states)} random states, {len(solved_states)} solved states")

False
False
False
False
True
True
True
True
False
False
False
False
False
True
True
True
False
False
False
False
False
True
True
True
True
False
False
False
False
False


KeyboardInterrupt: 

In [ ]:
# robot.get_ee_position() gets us the position of the robots "fingers"
# robot.get_fingers_width() gets us the width between the robots "fingers"

# the table has dimensions (1.1, 0.7, 0.4) w/ x_offset=-0.3
# can partition the table into 4 quadrants
# q1 = (1.1 / 4 - 0.3, 0.7 / 4 , 0.4)
# q2 = (1.1 / 4 - 0.3, 3 * (0.7 / 4), 0.4)
# q3 = ((1.1 / 4) * 3 - 0.3, 0.7 / 4, 0.4)
# q4 = ((1.1 / 4) * 3 - 0.3, 3 * (0.7 / 4), 0.4)

# distance_to_q{} = d(robot.get_ee_position(), q{})

# grasping might be distance from effectors to block & a check to see if they arent completely open or closed
# distance_to_block = d(robot.get_ee_position(), desired_goal)


In [ ]:
# task.get_achieved_goal() where the block is now
# desired_goal = np.array(sim.get_base_position("target"))
# d = distance(achieved_goal, desired_goal)

In [ ]:
# looks like overall we keep the env in the wrapped form and just save the variables
# so we can mess with them elsewhere